# Streaming Tables & Materialized Views in Databricks SQL

## Objectives
In this notebook, we will explore advanced Databricks SQL capabilities that bridge the gap between Data Warehousing and Data Engineering:

1.  **Streaming Tables (ST):** How to ingest data incrementally from cloud storage (S3/ADLS/Volumes) using SQL.
2.  **Materialized Views (MV):** How to create pre-computed aggregation tables that update incrementally.
3.  **The Backend:** Understand how DBSQL uses Serverless Delta Live Tables (DLT) pipelines to manage these objects.
4.  **Scheduling:** How to automate the refresh of these tables using SQL syntax.

## Prerequisites
*   **Unity Catalog** must be enabled.
*   You must use a **Serverless SQL Warehouse** or a **Pro SQL Warehouse** to execute these specific SQL commands (`CREATE STREAMING TABLE`, `MATERIALIZED VIEW`).
*   *Note:* While we are running this in a Notebook, these commands are native to the SQL Editor/DBSQL environment.

In [ ]:
# 1. SETUP: Create Dummy Data
# We need some files in a Volume to simulate an ingestion source.
# Let's create a Volume and write a CSV file to it.

import os

catalog = "dev"
schema = "bronze"
volume_name = "external_vol"
file_path = f"/Volumes/{catalog}/{schema}/{volume_name}/files/"

# Create directory if strictly local (for demo purposes) or verify volume exists
# dbutils.fs.mkdirs(file_path) # Uncomment if needed

# Define dummy data for Batch 1
data_batch_1 = """order_id,order_date,order_status,total_price
101,2024-01-01,O,150.00
102,2024-01-01,F,200.50
103,2024-01-02,O,120.00
104,2024-01-02,F,300.00
"""

# Write orders_1.csv
dbutils.fs.put(f"{file_path}orders_1.csv", data_batch_1, True)

print(f"Created orders_1.csv at {file_path}")

## 1. Creating a Streaming Table (ST)

A **Streaming Table** is a Delta table with extra support for streaming or incremental data processing.

We will use the function `read_files()` (a Table Valued Function) to act as a streaming source. It automatically detects new files added to the source directory.

**Key Syntax:**
```sql
CREATE OR REFRESH STREAMING TABLE <table_name> AS
SELECT * FROM STREAM read_files('<path>', format => 'csv')

In [ ]:
%sql
-- Create the Streaming Table
-- Note: Replace the path below with your specific Volume path if different
CREATE OR REFRESH STREAMING TABLE dev.bronze.orders_st
AS
SELECT *
FROM STREAM read_files(
  '/Volumes/dev/bronze/external_vol/files/',
  format => 'csv',
  header => 'true',
  schema => 'order_id INT, order_date DATE, order_status STRING, total_price DOUBLE'
);

### Verify Data Load
The table should now contain data from `orders_1.csv`.
**Note on Architecture:** When you ran the command above, Databricks actually kicked off a serverless DLT update in the background to load this data.

In [ ]:
%sql
SELECT * FROM dev.bronze.orders_st;

## 2. Incremental Loading (The "Streaming" part)

The power of a Streaming Table is that it tracks state. It knows which files it has already ingested. If we add a new file, running `REFRESH` will only process the **new** file.

Let's simulate arriving data by adding `orders_2.csv`.

In [ ]:
# Create Batch 2 Data
data_batch_2 = """order_id,order_date,order_status,total_price
105,2024-01-03,O,550.00
106,2024-01-03,F,100.00
"""

# Write orders_2.csv to the same folder
dbutils.fs.put(f"{file_path}orders_2.csv", data_batch_2, True)

print(f"Created orders_2.csv at {file_path}")

In [ ]:
%sql
-- Refresh the streaming table
-- This acts like a Trigger: AvailableNow in Spark Structured Streaming
REFRESH STREAMING TABLE dev.bronze.orders_st;

In [ ]:
%sql
-- Verify that new records (105, 106) are added
SELECT * FROM dev.bronze.orders_st;

## 3. Creating a Materialized View (MV)

A **Materialized View** computes and stores the result of a query. Unlike a standard View (which runs the query every time you call it), an MV stores the physical data.

**Key Benefit:** Performance. Complex aggregations are pre-computed.
**Incremental Refresh:** When the source (our Streaming Table) updates, the MV attempts to update *incrementally* rather than re-computing the whole table.

Let's create an MV to calculate total sales by status.

In [ ]:
%sql
CREATE OR REPLACE MATERIALIZED VIEW dev.bronze.orders_mv
AS
SELECT
  order_status,
  SUM(total_price) as agg_total_price,
  COUNT(*) as order_count
FROM dev.bronze.orders_st
GROUP BY order_status;

In [ ]:
%sql
-- Check the aggregated results
SELECT * FROM dev.bronze.orders_mv;

## 4. End-to-End Flow & Scheduling

Let's see the full power of the "Lakehouse" architecture.
1. We will add `orders_3.csv`.
2. We will `REFRESH` the Materialized View.
   * *Note:* Refreshing the MV automatically ensures upstream dependencies (the Streaming Table) are fresh. It cascades the update!

We can also add a `SCHEDULE` clause to automate this.

In [ ]:
# Create Batch 3 Data
data_batch_3 = """order_id,order_date,order_status,total_price
107,2024-01-04,O,90.00
"""

# Write orders_3.csv
dbutils.fs.put(f"{file_path}orders_3.csv", data_batch_3, True)
print(f"Created orders_3.csv")

In [ ]:
%sql
-- We can manually refresh the MV.
-- Because MV depends on ST, Databricks checks ST for new data (orders_3),
-- ingests it, and then updates the MV aggregation incrementally.

REFRESH MATERIALIZED VIEW dev.bronze.orders_mv;

### Scheduling Syntax
To make this run automatically (e.g., every hour), you modify the Create statement or use `ALTER`.

```sql
CREATE OR REPLACE MATERIALIZED VIEW dev.bronze.orders_mv
SCHEDULE CRON '0 0 * * * ?' -- Run every day at midnight (or use EVERY 1 HOUR)
AS
SELECT ...

In [ ]:
%sql
-- Example of altering schedule
-- This will ensure the table refreshes automatically every 4 hours
ALTER MATERIALIZED VIEW dev.bronze.orders_mv
SET SCHEDULE EVERY 4 HOURS;

## 5. Under the Hood: Delta Live Tables (DLT)

If you navigate to **Query History** in your Databricks Workspace after running these cells:
1.  Look for the `REFRESH` commands.
2.  You will see a link to a **Job/Pipeline**.
3.  Clicking that link takes you to a **Delta Live Tables** pipeline interface.

**Insight:**
Databricks SQL uses a "Serverless DLT" backend to manage the state, checkpoints, and incremental processing logic for Streaming Tables and Materialized Views.
*   **No-Op:** If no new files are found, the pipeline creates a "No-Op" (No Operation) event, saving compute cost.
*   **Group Aggregate:** For MVs, it uses stateful aggregation to only update the changed groups.

In [ ]:
# Clean up (Optional)
# dbutils.fs.rm(file_path, True)
# spark.sql("DROP MATERIALIZED VIEW IF EXISTS dev.bronze.orders_mv")
# spark.sql("DROP TABLE IF EXISTS dev.bronze.orders_st")